In [ ]:
import pandas as pd
import numpy as np
import gc
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

print('loading files ...')

prior_orders = pd.read_csv('data/order_products__prior.csv', dtype={'order_id': np.uint32,
           'product_id': np.uint16, 'reordered': np.uint8, 'add_to_cart_order': np.uint8})

train_orders = pd.read_csv('data/order_products__train.csv', dtype={'order_id': np.uint32,
           'product_id': np.uint16, 'reordered': np.int8, 'add_to_cart_order': np.uint8 })

orders = pd.read_csv('data/orders.csv', dtype={'order_hour_of_day': np.uint8,
           'order_number': np.uint8, 'order_id': np.uint32, 'user_id': np.uint32,
           'order_dow': np.uint8, 'days_since_prior_order': np.float16})

orders.eval_set = orders.eval_set.replace({'prior': 0, 'train': 1, 'test':2}).astype(np.uint8)
orders.days_since_prior_order = orders.days_since_prior_order.fillna(30).astype(np.uint8)

products = pd.read_csv('data/products.csv', dtype={'product_id': np.uint16,
            'aisle_id': np.uint8, 'department_id': np.uint8},
             usecols=['product_id', 'aisle_id', 'department_id'])
products1 = products.set_index('product_id')
train_orders["product_id"] = train_orders["product_id"].astype(str)
prior_orders["product_id"] = prior_orders["product_id"].astype(str)
train_products = train_orders.groupby("order_id").apply(lambda order: order['product_id'].tolist())
prior_products = prior_orders.groupby("order_id").apply(lambda order: order['product_id'].tolist())
sentences = prior_products.append(train_products)
longest = np.max(sentences.apply(len))
sentences = sentences.values

import gensim

model = gensim.models.Word2Vec.load("data/product2vec.model")
vocab = list(model.wv.vocab.keys())

from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca.fit(model.wv.syn0)

pca_result = pca.fit_transform(model.wv.syn0)
pca_result_df = pd.DataFrame(pca_result, index = vocab)

pca_result_df = pca_result_df.reset_index()
pca_result_df.columns = ["product_id", 0, 1, 2]
pca_result_df.to_csv("data/pca_w2v.csv", index=False)
pca_result_df.head()